# Data Preparation

The aim of this notebook is to extract the data, clean it, process it, merge the different data sets and output the data that will be finally used for the modelling.  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time

import requests # to read the data from the REST API of EPA Envirofacts
import csv # needed as data accessed through the REST API are .csv (other possibilities are .xml or .xls)


## Extracting Water Systems Data

In [26]:
# more on the API: https://www.epa.gov/enviro/envirofacts-data-service-api

# notes on the API:
#     - WATER_SYSTEM = table name
#     - PWS_ACTIVITY_CODE/A ==> select only active water systems
#     - EPA_REGION/01 ==> New England

CSV_URL = 'https://enviro.epa.gov/enviro/efservice/WATER_SYSTEM/EPA_REGION/01/PWS_ACTIVITY_CODE/A/CSV'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    initial_WS = list(cr)        
    WATER_SYSTEM_raw = pd.DataFrame(initial_WS)
print(WATER_SYSTEM_raw.shape)
WATER_SYSTEM_raw.head()

(10483, 48)


,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,WATER_SYSTEM.PWSID,WATER_SYSTEM.PWS_NAME,WATER_SYSTEM.NPM_CANDIDATE,WATER_SYSTEM.PRIMACY_AGENCY_CODE,WATER_SYSTEM.EPA_REGION,WATER_SYSTEM.SEASON_BEGIN_DATE,WATER_SYSTEM.SEASON_END_DATE,WATER_SYSTEM.PWS_ACTIVITY_CODE,WATER_SYSTEM.PWS_DEACTIVATION_DATE,WATER_SYSTEM.PWS_TYPE_CODE,...,WATER_SYSTEM.ZIP_CODE,WATER_SYSTEM.COUNTRY_CODE,WATER_SYSTEM.STATE_CODE,WATER_SYSTEM.SOURCE_WATER_PROTECTION_CODE,WATER_SYSTEM.SOURCE_PROTECTION_BEGIN_DATE,WATER_SYSTEM.OUTSTANDING_PERFORMER,WATER_SYSTEM.OUTSTANDING_PERFORM_BEGIN_DATE,WATER_SYSTEM.CITIES_SERVED,WATER_SYSTEM.COUNTIES_SERVED,
1,ME0004628,MACHIAS TRAILER PARK,Y,ME,01,,,A,,CWS,...,04654,US,ME,N,,,,MACHIAS,Washington,None
2,ME0092288,MARSH BROOK ESTATES,Y,ME,01,,,A,,CWS,...,01746,US,MA,N,,,,SANFORD,York,None
3,ME0009198,TRAILS END STEAK HOUSE & TAVERN,Y,ME,01,01-01,12-31,A,,TNCWS,...,04936,US,ME,N,,,,EUSTIS,Franklin,None
4,ME0094505,R & R VACATION HOME PARK,Y,ME,01,01-01,12-31,A,,TNCWS,...,04055,US,ME,N,,,,NAPLES,Cumberland,None


In [27]:
# Some Data Cleaning:

water_system = WATER_SYSTEM_raw.copy()

# set the first row as header:
new_header = water_system.iloc[0] # grab the first row for the header
new_header = new_header.str.split('.').str[1] # we remove the redundant table name (WATER_SYSTEM) in column names
new_header = new_header.str.lower() # set to lower case, as less annoying
water_system = water_system[1:] # take the data less the header row
water_system.columns = new_header # set the header row as the df header

# we remove the last column of null, that is an artifact of the extraction:
water_system = water_system.dropna(axis = 1, how='all') # axis = 1 = columns

water_system.tail() # looks good for now.

,pwsid,pws_name,npm_candidate,primacy_agency_code,epa_region,season_begin_date,season_end_date,pws_activity_code,pws_deactivation_date,pws_type_code,...,city_name,zip_code,country_code,state_code,source_water_protection_code,source_protection_begin_date,outstanding_performer,outstanding_perform_begin_date,cities_served,counties_served
10478,NH1108030,WINDY RIDGE ORCHARD,Y,NH,01,06-01,10-31,A,,TNCWS,...,HAVERHILL,03774,US,NH,,,,,HAVERHILL,Grafton
10479,NH1109020,MOUNTAIN VALLEY TREATMENT CTR,Y,NH,01,01-01,12-31,A,,TNCWS,...,ORFORD,03777,US,NH,,,,,HAVERHILL,Grafton
10480,NH1112010,STONEGATE ACRES,Y,NH,01,,,A,,CWS,...,CONCORD,03302,US,NH,,,,,HEBRON,Grafton
10481,NH1113010,HILLSIDE INN CONDOS,Y,NH,01,,,A,,CWS,...,HEBRON,03241,US,NH,,,,,HEBRON,Grafton
10482,NH1117010,CAMP BEREA/DINING HALL,Y,NH,01,01-01,12-31,A,,TNCWS,...,HEBRON,03241,US,NH,,,,,HEBRON,Grafton
